In [1]:
from plotly.subplots import make_subplots
import pandas as pd
import plotly.graph_objs as go
import sqlite3

C:\Users\37061\AppData\Local\Temp\ipykernel_24372\583827411.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Connect to SQLite database
conn = sqlite3.connect('energy.db')

# Source: Our World in Data
df_share_electricity_imports = pd.read_sql_query('SELECT * FROM "electricity-imports-share-demand (OWD)"', conn)
df_critical_materials_imports = pd.read_sql_query('SELECT * FROM "Critical mineral imports by country (OWD)"', conn)
df_critical_materials_exports = pd.read_sql_query('SELECT * FROM "Critical mineral exports by country (OWD)"', conn)

# Source: International energy organization
df_scenarios = pd.read_sql_query('SELECT * FROM "Critical minerals (IEA) all scenarios"', conn)

conn.close()

In [3]:
eu_countries = ['Germany', 'France', 'Italy', 'Spain', 'Netherlands', 'Belgium', 'Poland',
                'Romania', 'Greece', 'Portugal', 'Czech Republic', 'Hungary', 'Sweden',
                'Austria', 'Bulgaria', 'Denmark', 'Finland', 'Slovakia', 'Ireland', 'Croatia',
                'Lithuania', 'Slovenia', 'Estonia', 'Latvia', 'Cyprus', 'Luxembourg', 'Malta']


entities_list = ['Asia', 'Austria', 'North America', 'Africa']
combined_entities = entities_list + eu_countries
combined_filtered_data = df_share_electricity_imports[df_share_electricity_imports['Entity'].isin(combined_entities)]

regions_data = combined_filtered_data[combined_filtered_data['Entity'].isin(entities_list)]

# Aggregate energy imports for the EU countries
eu_imports = combined_filtered_data[combined_filtered_data['Entity'].isin(eu_countries)]
eu_combined_imports = eu_imports.groupby('Year')['Net electricity imports as a share of demand - %'].sum().reset_index()

fig19 = go.Figure()

for entity in entities_list:
    region_data = regions_data[regions_data['Entity'] == entity]
    fig19.add_trace(go.Scatter(x=region_data['Year'], y=region_data['Net electricity imports as a share of demand - %'], 
                             mode='lines', name=entity))


fig19.add_trace(go.Scatter(x=eu_combined_imports['Year'], y=eu_combined_imports['Net electricity imports as a share of demand - %'], 
                         mode='lines', name='EU'))

fig19.update_layout(title='Net electricity imports as a share of demand - %',
    xaxis_title='Year',
    yaxis_title='Electricity Imports (share of demand - %)',
    showlegend=True, 
    height=600,
    template="plotly_dark")

fig19.add_annotation(
    text="Source: Our World in Data 2024",
    xref="paper", yref="paper",
    x=0, y=-0.15,
    showarrow=False,
    font=dict(size=10, color="grey")
)

fig19.show()

In [4]:
# Melt the DataFrame to have one column for year and one column for export values
df_melted = df_critical_materials_exports.melt(id_vars=['Mineral', 'Country'], var_name='Year', value_name='Export_Value')

# Rename 
df_melted['Mineral'] = df_melted['Mineral'].replace({'PGM, rare earth and other ': 'Platinum Group Metals'})

# Filter data for the year 2022
df_2022 = df_melted[df_melted['Year'] == '2022']

# Group by mineral
grouped = df_2022.groupby('Mineral')

top_exporters_2022 = {}
for mineral, group in grouped:
    top_exporters_2022[mineral] = group.nlargest(5, 'Export_Value')

# Create traces for each mineral
data = []
for mineral, top in top_exporters_2022.items():
    data.append(go.Bar(x=top['Country'], y=top['Export_Value'], name=mineral, opacity=0.7))

layout = go.Layout(
    title='Top Exporters for Critical Minerals in 2022',
    xaxis=dict(title='Country'),
    yaxis=dict(title='Export Value (billion USD)')
)

fig20 = go.Figure(data=data, layout=layout)

fig20.add_annotation(
    text="Source: Our World in Data 2024",
    xref="paper", yref="paper",
    x=0, y=-0.5,
    showarrow=False,
    font=dict(size=10, color="grey")
)
fig20.update_layout(template="plotly_dark")
fig20.show()


In [5]:
# One column for mineral and country, one column for year, and one column for import value
df_melted_imports = df_critical_materials_imports.melt(id_vars=['Mineral', 'Country'], var_name='Year', value_name='Import_Value')

# Filter data for the year 2022
df_2022_imports = df_melted_imports[df_melted_imports['Year'] == '2022']

# Convert to numeric 
df_2022_imports['Import_Value'] = pd.to_numeric(df_2022_imports['Import_Value'], errors='coerce')

# Rename 
df_2022_imports['Mineral'] = df_2022_imports['Mineral'].replace({'PGM, rare earth and other ': 'Platinum Group Metals'})
df_2022_imports['Country'] = df_2022_imports['Country'].replace('Korea, Republic of', 'South Korea')

# Group the data by mineral
grouped_imports = df_2022_imports.groupby('Mineral')
top_importers_2022 = {}
for mineral, group in grouped_imports:
    top_importers_2022[mineral] = group.nlargest(5, 'Import_Value')
    
# Create traces for each mineral
data_imports = []
for mineral, top in top_importers_2022.items():
    data_imports.append(go.Bar(x=top['Country'], y=top['Import_Value'], name=mineral, opacity=0.7))

layout_imports = go.Layout(
    title='Top Importers for Critical Minerals in 2022',
    xaxis=dict(title='Country'),
    yaxis=dict(title='Import Value (Millions USD)')
)

fig21 = go.Figure(data=data_imports, layout=layout_imports)

fig21.add_annotation(
    text="Source: Our World in Data 2024",
    xref="paper", yref="paper",
    x=0, y=-0.15,
    showarrow=False,
    font=dict(size=10, color="grey")
)
fig21.update_layout(template="plotly_dark")
fig21.show()

C:\Users\37061\AppData\Local\Temp\ipykernel_24372\2188028388.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\37061\AppData\Local\Temp\ipykernel_24372\2188028388.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\37061\AppData\Local\Temp\ipykernel_24372\2188028388.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

In [6]:
import plotly.graph_objects as go

minerals = ['Copper', 'Cobalt', 'Lithium', 'Nickel', 'Neodymium']

# Colors for scenarios
scenario_colors = ['blue', 'orange', 'green'] 

# Create a plot for each mineral
for i, mineral in enumerate(minerals, start=22):  # Start numbering from 23
    mineral_demand = df_scenarios[(df_scenarios['Mineral'] == mineral) & (df_scenarios['Industry'] == 'Total demand')]
    scenarios = mineral_demand['Scenario']

    fig = go.Figure()

    for j, scenario in enumerate(scenarios, start=1):
        fig.add_trace(
            go.Bar(
                x=df_scenarios.columns[3:],
                y=mineral_demand.iloc[j-1, 3:],
                name=scenario,
                marker=dict(color=scenario_colors[j-1])  # Assign color 
            )
        )

    fig.update_layout(
        title="Predicted Demand in Different Scenarios Over the Years for " + mineral,
        xaxis_title="Year",
        yaxis_title="Demand",
        template="plotly_dark",
        annotations=[
            dict(
                text="Source: IEA Critical Minerals Market Review 2023",
                xref="paper", yref="paper",
                x=0, y=-0.15,
                showarrow=False,
                font=dict(size=10, color="grey")
            )
        ],
        showlegend=True
    )

    locals()["fig" + str(i)] = fig  # Assign the figure to a variable with the name "fig" + i

# Now you can show each figure using the corresponding variable name

fig22.show()
fig23.show()
fig24.show()
fig25.show()
fig26.show()




In [7]:
import plotly.express as px
from jinja2 import Template
import json

figs = [fig19, fig20, fig21, fig22, fig23, fig24, fig25, fig26]  

# Convert each Plotly figure to JSON
fig_jsons = [fig.to_json() for fig in figs]

# Load the Jinja2 template
input_template_path = "html_samples/input.html"
with open(input_template_path) as template_file:
    j2_template = Template(template_file.read())
rendered_html = j2_template.render(fig_jsons=fig_jsons)

# Save HTML
output_html_path = "html_samples/Global.html"
with open(output_html_path, "w", encoding="utf-8") as output_file:
    output_file.write(rendered_html)
